<a href="https://colab.research.google.com/github/recife25/AGSAdvGenAIBuildingLLMAppProject/blob/main/AI_Powered_HR_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Felipe Ruiz

===AI Power HR Assistant Project===

AGS Advance Generative AI: Building LLM app

Nestle hr policy pdf file

gradio doc init

course project ai powered hr assistant

In [ ]:
pip install openai gradio PyPDF2 faiss-cpu tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 49.1 MB/s eta 0:00:00


In [29]:
#code sets the openai api key - which is secret - setting up programming env
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [21]:
#PyPDF2 library helps with the extraction of pdf files - programming env
import PyPDF2

In [22]:
#extract the nestle org pdf file - processing text documents - the pdf file
def extract_text_from_pdf(file_path):
  text = ""
  with open(file_path, "rb") as file:
    reader = PyPDF2.PdfReader(file)
    for page in reader.pages:
      text += page.extract_text()
  return text

In [23]:
#split the Text into chunks - cont processing text documents - the chunks
def split_text(text, max_tokens=500):
    sentences = text.split('. ')
    chunks, current_chunk = [], ''
    for sentence in sentences:
        if len(current_chunk.split()) + len(sentence.split()) <= max_tokens:
            current_chunk += sentence + '. '
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + '. '
    chunks.append(current_chunk.strip())
    return chunks


In [24]:
#embed the text - setting chunks into sets prior to vectoring - the lego parts

import openai

def get_embedding(text, model="text-embedding-ada-002"):
    client = openai.OpenAI()
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

In [25]:
#create the vector store - the vector representation
import faiss
import numpy as np

def build_faiss_index(chunks):
    embeddings = [get_embedding(chunk) for chunk in chunks]
    dim = len(embeddings[0])
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings).astype('float32'))
    return index, embeddings


In [26]:
#build the search + answer system - the Q&A ask me anything ?
import faiss
import numpy as np
import openai

def search_and_respond(user_query, chunks, index, embeddings, top_k=1):
    query_embedding = get_embedding(user_query)
    D, I = index.search(np.array([query_embedding]).astype('float32'), top_k)
    context = " ".join([chunks[i] for i in I[0]])

    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": f"Context: {context}\n\nUser Question: {user_query}\n\nAnswer:"}]
    )
    return response.choices[0].message.content.strip()

In [27]:
#Replace with the actual path to your PDF file - forgot pdf file needs colab files path
file_path = '/content/1728286846_the_nestle_hr_policy_pdf_2012.pdf'
text = extract_text_from_pdf(file_path)
chunks = split_text(text)
index, embeddings = build_faiss_index(chunks)

In [28]:
#create gradio chat interface - open the pdf file briefly read it , and follow with
#Q&A session using the Nestle HR Assistant
import gradio as gr

def chatbot_interface(query):
    return search_and_respond(query, chunks, index, embeddings)

iface = gr.Interface(fn=chatbot_interface, inputs="text", outputs="text", title="Nestlé HR Assistant")
iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://75b2a1c0ca42890eee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


sample Q&A exchange:
Felipe Ruiz
Project AI-Powered-HR-Assistant

======

section: introduction
provide me with a brief summary of keypoints in the letter to nestle hr by Jean-Marc Duvoisin Deputy Executive Vice President

output:
In the introduction of the Nestlé Human Resources Policy, Jean-Marc Duvoisin emphasizes the importance of employees to the success of the company. He highlights the role of line managers in creating a supportive work environment and developing future leaders. Duvoisin outlines the responsibilities of HR managers in guiding line managers to optimize employee performance and achieve business goals. He also mentions the HR structure based on specialized services, business partnerships, and employee services, all working towards the common goal of ensuring the long-term success of the company through the effective management and development of employees.


======

Can you outline key points of these letter, provide me with numberic bullets

output:
1. The Nestlé Human Resources Policy is a mandatory policy for all employees, approved by the Executive Board of Nestlé S.A. in September 2012.
2. The policy emphasizes the importance of employees as the key to Nestlé's success and outlines guidelines for effective Human Resources Management globally.
3. It is based on the Nestlé Management and Leadership Principles and Corporate Business Principles.
4. Line managers are responsible for creating an environment where employees are committed to their work and contribute to the company's success.
5. HR managers support line managers in aligning business needs with people requirements to optimize performance and ensure exemplary working conditions.
6. The HR structure includes Centres of Expertise, Business Partners, and Employee Services to provide specialized services and streamline HR processes.
7. The success of the Company depends on attracting, retaining, and developing employees for ongoing and sustainable growth.

======

page one has in the upper right corner , the nestle logo for good food , good life , can you describe what are the elements of this logo?


The Nestlé logo for "good food, good life" typically includes a stylized representation of a nest with several small birds inside, symbolizing nurturing and care. The logo may also include the Nestlé name in a distinctive font, along with the company's tagline. The colors of the logo are often a combination of blue, white, and red, reflecting the brand's corporate identity.

======

section: Employment and working conditions , does the nestle hr policy address any issues concerning the legal rights of employees to form a labor union?

output:
Based on the provided information, the Nestlé Human Resources Policy does not explicitly mention any issues concerning the legal rights of employees to form a labor union. The policy primarily focuses on the importance of employee engagement, the role of line managers in creating a positive work environment, and the overall objectives of the Human Resources function within Nestlé. It does not specifically address the topic of labor unions or the legal rights of employees to form them.

=====
interesting interaction here:
on page 4 there two main topic in red color , for example , A shared responsibility and Joining Nestle can you see that ?

output:
I'm sorry, but as an AI text-based model, I am unable to see specific colors or formatting such as red color in the text. However, based on the information provided in the context, it seems like the main topics highlighted are "A shared responsibility" and "Joining Nestle." These topics emphasize the importance of individual responsibility, teamwork, and collaboration within Nestle as an organization.


from page 4 to page 7 , each page has 2 main topics highlighted , can you tell me what those main topics highlighted are from pages 4 to 7

output:

Sure, I can provide you with the main topics highlighted on pages 4 to 7 of the document you provided. However, since I cannot directly see the content of the document, I will do my best to give you an accurate response based on the information you have provided.

Page 4:
1. Business requirements for an agile and innovative company
2. Strong willingness to support others and work in multi-skilled teams

Page 5:
1. Dynamic organization creating a climate of innovation
2. Encouragement of taking risks and willingness to correct and learn from mistakes

Page 6:
1. Combining scope and brand strength of a global company with creativity and knowledge of a local business
2. Far-reaching influence and exploration of full long-term potential

Page 7:
1. Corporate policy - Nestlé on the Move: A flexible and dynamic organization
2. Collaboration and support from line managers and employees.




======

on page 4, A shared responsibility and Joining Nestle , can you called those "headers"  

output:
Yes, "A shared responsibility" and "Joining Nestle" can be called headers on page 4.